# Using Little VGG for emotion detection

Training Emotion Detector

In [1]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization,Conv2D,MaxPooling2D
import os

num_classes=6
img_rows,img_cols=48,48
batch_size=18

train_data_dir='./fer2013/train'
validation_data_dir='./fer2013/validation'

#data augmentation
train_datagen=ImageDataGenerator(
             rescale=1./255,
             rotation_range=30,
             shear_range=0.5,
             width_shift_range=0.6,
             height_shift_range=0.3,
             horizontal_flip=True,
             fill_mode='nearest')

validation_datagen=ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(
            train_data_dir,
            color_mode='grayscale',
            target_size=(img_rows,img_cols),
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=True)

validation_generator=validation_datagen.flow_from_directory(
            validation_data_dir,
            color_mode='grayscale',
            target_size=(img_rows,img_cols),
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=True)


Found 28273 images belonging to 6 classes.
Found 3534 images belonging to 6 classes.
